In [1]:
#2018 HMDA Edit Testing File Generator
from collections import OrderedDict
from io import StringIO
import json
import os
import pandas as pd
import random

#custom imports
import lar_constraints
import lar_generator
from rules_engine import rules_engine
from test_file_generator import test_data
import utils
!pwd

/Users/roellk/Desktop/HMDA/hmda-test-files/python


In [2]:
#2018 Filing Instruction Guide: https://www.consumerfinance.gov/data-research/hmda/static/for-filers/2018/2018-HMDA-FIG.pdf

use_cols = ['name', 'metDivName', 'countyFips', 'geoIdMsa', 'metDivFp', 'smallCounty', 'tracts']
cbsa_cols = ['name', 'metDivName', 'state', 'countyFips', 'county', 'tracts','geoIdMsa', 'metDivFp', 'smallCounty', 
             'stateCode', 'tractDecimal']
cbsas = pd.read_csv('../dependancies/tract_to_cbsa_2015.txt', usecols=use_cols, delimiter='|', 
                    header=None, names=cbsa_cols, dtype=str) #load tract to CBSA data from platform file
cbsas["tractFips"] = cbsas.countyFips + cbsas.tracts
counties = list(cbsas.countyFips)
tracts = list(cbsas.tractFips)

In [3]:
#load schemas for LAR and transmittal sheet
lar_schema_df = pd.DataFrame(json.load(open("../schemas/lar_schema.json", "r")))
ts_schema_df = pd.DataFrame(json.load(open("../schemas/ts_schema.json", "r")))

In [4]:
lar_gen = lar_generator.lar_gen(lar_schema_df, ts_schema_df, counties=counties, tracts=tracts) #instantiate generator
lar_const = lar_constraints.lar_constraints(counties=counties, tracts=tracts)#instantiate constraints
lar_validator = rules_engine(lar_schema=lar_schema_df, ts_schema=ts_schema_df, tracts=tracts, counties=counties)

In [5]:
#Set parameters for data creation
file_length = 5 #set number of rows in test file
first = True
lei = None

In [6]:
#Create a sample TS row
#set dummy values for TS row
def make_ts_row(file_length, lei):
    ts_row = OrderedDict()
    ts_row["record_id"]="1"
    ts_row["inst_name"]="Ficus Bank"
    ts_row["calendar_year"]= "2018"
    ts_row["calendar_quarter"]="4"
    ts_row["contact_name"]="Mr. Smug Pockets"
    ts_row["contact_tel"]="555-555-5555"
    ts_row["contact_email"]="pockets@ficus.com"
    ts_row["contact_street_address"]="1234 Ficus Lane"
    ts_row["office_city"]="Ficusville"
    ts_row["office_state"]="UT"
    ts_row["office_zip"]="84096"
    ts_row["federal_agency"]="9"
    ts_row["lar_entries"]= str(file_length)
    ts_row["tax_id"]="01-0123456"
    ts_row["lei"]=str(lei)
    return ts_row

In [7]:
def get_const_list():
    """Creates a list of constraints from the functions in the lar_constraints object."""
    constraints = [] 
    for func in dir(lar_const):
        if func[:1] in ("s", "v") and func[1:4].isdigit()==True:
            constraints.append(func)
    return constraints
            
def constraints_loop(constraints=[], row=None, row_base=None):
    for const in constraints:
        row = apply_constraint(row, const)
        diff = get_diff(row, row_base)
    return row

def apply_constraint(row, func):
    """Applies all constraints in the constrains list and returns a LAR row in dictionary format."""
    row_start = row.copy()
    row = getattr(lar_const, func)(row) #apply constraint to row
    diff_1, diff_2 = get_diff(row, row_start)
    if len(diff_1) > 0:
        print(str(func))
        print(diff_1, "\n\n", diff_2)
    return row

def get_diff(row, row_base):
    """Checks the difference between an initial row and the row after constraints are applied"""
    initial_row = set(row_base.items()) #convert initial row to set
    changed_row = set(row.items()) #convert constrained row to set
    diff_1 = (changed_row - initial_row) #subtract row sets to show changes from constraint funcs
    diff_2 = (initial_row - changed_row)
    return diff_1, diff_2

def validation(row, ts_row):
    """"""
    lar_data = pd.DataFrame(row, index=[1])
    ts_data = pd.DataFrame(ts_row, index=[0])
    rules_check = rules_engine(lar_schema=lar_schema_df, ts_schema=ts_schema_df, tracts=tracts, 
                             counties=counties) #instantiate edits rules engine
    rules_check.load_lar_data(lar_data)
    rules_check.load_ts_data(ts_data)
    for func in dir(rules_check):
        if func[:1] in ("s", "v") and func[1:4].isdigit()==True:
            #print("applying:", func)
            getattr(rules_check, func)()
    return rules_check.results

def write_file(ts_input=None, lar_input=None, directory="../edits_files/", name="passes_all.txt"):
    """Takes a TS row as a dictionary and LAR data as a dataframe. Writes LAR data to file and 
    re-reads it to combine with TS data to make a full file."""
    #make directories for files if they do not exist
    if not os.path.exists(directory):
        os.makedirs(directory)

    #write LAR dataframe to file
    parts_dir = directory+"file_parts/"
    if not os.path.exists(parts_dir):
        os.makedirs(parts_dir)
        
    lar_input.to_csv(parts_dir + "lar_data.txt", sep="|", header=False, index=False, index_label=False)
    #load LAR data as file rows
    with open(parts_dir + "lar_data.txt", 'r') as lar_data:
        lar = lar_data.readlines()

    with open(directory + name, 'w') as final_file:
        final_file.write("|".join(ts_input.values())+"\n")
        for line in lar:
            final_file.write("{line}".format(line=line))

In [8]:
first = True
for i in range(0, file_length): #loop over file length
    print("making new row {row_num}\n\n".format(row_num=i), "*"*50)
    if lei:
        row = lar_gen.make_row(lei=lei) #generate new LEI. The same LEI must be used for each row
    else:
        row = lar_gen.make_row() #create new row
        
    lei = row["lei"] #copy LEI from previous row
    iters = 1 #start iteration count for checking diff time

    ts_row = make_ts_row(file_length, lei) #create dictionary of TS row fields
    stop = False
    #flag for starting the LAR dataframe
    while stop == False:
        row_base = row.copy() #copy row to enable diff
        res = pd.DataFrame(validation(row, ts_row))
        print(res[res.status=="failed"])
        if len(res[res.status=="failed"])<=0:
            stop = True
        else:
            print("\nstarting constraints iteration {iter}".format(iter=iters))
            row = constraints_loop(get_const_list(), row, row_base)
        iters+=1
    
    if first: #create first row of dataframe
        lar_frame = pd.DataFrame(row, index=[1])
        first = False
        print("finished row\n")
    else: #add additional rows to dataframe
        #print("concating")
        print("finished row\n")
        new_lar = pd.DataFrame(row, index=[1])
        lar_frame = pd.concat([lar_frame, new_lar], axis=0)
        
#lar_frame.reset_index(inplace=True) #reset index
#lar_frame.drop("index", inplace=True, axis=1) #drop additional index column
print("LAR dataframe complete")

making new row 0

 **************************************************
    edit_name  fail_count                      fields  \
24     v612_2         1.0                loan_purpose   
29     v614_1         1.0                 preapproval   
30     v614_2         1.0                 preapproval   
31     v614_3         1.0                 preapproval   
32     v614_4         1.0                 preapproval   
34     v615_2         1.0                const_method   
35     v615_3         1.0                const_method   
41     v619_3         1.0                 action_date   
58     v628_3         1.0       applicant ethnicities   
68     v631_4         1.0          Co-App Ethnicities   
70     v632_2         1.0      Co-App Ethnicity Basis   
74       v634         1.0      Co-App Ethnicity Basis   
75     v635_1         1.0                  App Race 1   
81     v636_3         1.0        Applicant Race Basis   
86     v638_3         1.0          Co-Applicant Races   
89     v639_2     

    edit_name  fail_count              fields  \
21     v610_2         1.0            app_date   
145    v669_4         1.0  Denial Reasons 1-4   
221    v700_1         1.0     AUS and Results   
222    v700_2         1.0     AUS and Results   

                                             row_ids row_type  status  
21   [0PH53M7NA5F79002BWX8TJ0E017DX6HFRJT4AZ9WM7D20]      LAR  failed  
145  [0PH53M7NA5F79002BWX8TJ0E017DX6HFRJT4AZ9WM7D20]      LAR  failed  
221  [0PH53M7NA5F79002BWX8TJ0E017DX6HFRJT4AZ9WM7D20]      LAR  failed  
222  [0PH53M7NA5F79002BWX8TJ0E017DX6HFRJT4AZ9WM7D20]      LAR  failed  

starting constraints iteration 2
v610_const
{('app_date', 'NA')} 

 {('app_date', '20180627')}
v669_const
{('denial_2', ''), ('denial_3', ''), ('denial_4', '')} 

 {('denial_3', '5'), ('denial_2', '9'), ('denial_4', '6')}
v671_const
{('denial_code_9', '')} 

 {('denial_code_9', 'DQ6QJDGJGW6QHVSAPK99FMGC6POZ9IIKVPR49YJDB7FQXRNNM448D7KGXG9BNGFEP2ODOGXQKRK6H29PBD9ZP97YWMR10GMCZGF2N3QY0R0U53GC0

    edit_name  fail_count              fields  \
77     v635_3         1.0     Applicant Races   
87     v638_4         1.0  Co-Applicant Races   
145    v669_4         1.0  Denial Reasons 1-4   

                                             row_ids row_type  status  
77   [0PH53M7NA5F79002BWX8T8VECSQYPUJ4VMPOSDWTVTB66]      LAR  failed  
87   [0PH53M7NA5F79002BWX8T8VECSQYPUJ4VMPOSDWTVTB66]      LAR  failed  
145  [0PH53M7NA5F79002BWX8T8VECSQYPUJ4VMPOSDWTVTB66]      LAR  failed  

starting constraints iteration 2
v635_3_const
{('app_race_5', '44')} 

 {('app_race_5', '1')}
v636_const
{('app_race_5', '2')} 

 {('app_race_5', '44')}
v638_const
{('co_app_race_4', ''), ('co_app_race_2', ''), ('co_app_race_5', ''), ('co_app_race_3', '')} 

 {('co_app_race_3', '24'), ('co_app_race_5', '3'), ('co_app_race_2', '25'), ('co_app_race_4', '43')}
v669_const
{('denial_2', ''), ('denial_3', ''), ('denial_4', '')} 

 {('denial_4', '1'), ('denial_3', '3'), ('denial_2', '7')}
Empty DataFrame
Columns: [e

    edit_name  fail_count              fields  \
21     v610_2         1.0            app_date   
87     v638_4         1.0  Co-Applicant Races   
221    v700_1         1.0     AUS and Results   
222    v700_2         1.0     AUS and Results   

                                             row_ids row_type  status  
21   [0PH53M7NA5F79002BWX8VPWODHPHWGO55LV0TUHCU6H76]      LAR  failed  
87   [0PH53M7NA5F79002BWX8VPWODHPHWGO55LV0TUHCU6H76]      LAR  failed  
221  [0PH53M7NA5F79002BWX8VPWODHPHWGO55LV0TUHCU6H76]      LAR  failed  
222  [0PH53M7NA5F79002BWX8VPWODHPHWGO55LV0TUHCU6H76]      LAR  failed  

starting constraints iteration 2
v610_const
{('app_date', 'NA')} 

 {('app_date', '20180510')}
v638_const
{('co_app_race_4', ''), ('co_app_race_5', ''), ('co_app_race_2', '')} 

 {('co_app_race_2', '41'), ('co_app_race_5', '3'), ('co_app_race_4', '21')}
v700_const
{('aus_result_3', ''), ('aus_4', ''), ('aus_result_4', ''), ('aus_2', ''), ('aus_result_2', ''), ('aus_3', '')} 

 {('aus_result

   edit_name  fail_count           fields  \
77    v635_3         1.0  Applicant Races   

                                            row_ids row_type  status  
77  [0PH53M7NA5F79002BWX8RCV5I1L19TOVIGHJW24XGDA79]      LAR  failed  

starting constraints iteration 2
v635_3_const
{('app_race_4', '24')} 

 {('app_race_4', '5')}
v636_const
{('app_race_4', '5')} 

 {('app_race_4', '24')}
   edit_name  fail_count           fields  \
77    v635_3         1.0  Applicant Races   

                                            row_ids row_type  status  
77  [0PH53M7NA5F79002BWX8RCV5I1L19TOVIGHJW24XGDA79]      LAR  failed  

starting constraints iteration 3
v635_3_const
{('app_race_4', '2'), ('app_race_5', '4')} 

 {('app_race_4', '5'), ('app_race_5', '2')}
Empty DataFrame
Columns: [edit_name, fields, row_type, status]
Index: []
finished row

making new row 4

 **************************************************
    edit_name  fail_count                      fields  \
21     v610_2         1.0     

    edit_name  fail_count              fields  \
87     v638_4         1.0  Co-Applicant Races   
145    v669_4         1.0  Denial Reasons 1-4   

                                             row_ids row_type  status  
87   [0PH53M7NA5F79002BWX8UXE817LF67RTWMF6Y9F9DVW79]      LAR  failed  
145  [0PH53M7NA5F79002BWX8UXE817LF67RTWMF6Y9F9DVW79]      LAR  failed  

starting constraints iteration 2
v638_const
{('co_app_race_2', ''), ('co_app_race_3', '')} 

 {('co_app_race_2', '4'), ('co_app_race_3', '21')}
v669_const
{('denial_2', ''), ('denial_3', ''), ('denial_4', '')} 

 {('denial_2', '5'), ('denial_4', '7'), ('denial_3', '8')}
Empty DataFrame
Columns: [edit_name, fields, row_type, status]
Index: []
finished row

LAR dataframe complete


In [9]:
#check validity and syntax of data using rules_engine
#instantiate edits rules engine
validator = rules_engine(lar_schema=lar_schema_df, ts_schema=ts_schema_df, tracts=tracts, counties=counties) 
#load data to validator engine
validator.load_lar_data(lar_frame)
validator.load_ts_data(pd.DataFrame(ts_row, index=[0], columns=validator.ts_field_names))
#check data
for func in dir(validator):
    if func[:1] in ("s", "v") and func[1:4].isdigit()==True:
        #print("applying:", func)
        getattr(validator, func)()

#get validation results
results_df = pd.DataFrame(validator.results)
results_df[results_df.status=="failed"]

,edit_name,fields,row_type,status


In [10]:
#Quality and Macro field interrelationship constraints:

In [11]:
#write sample file to disk
write_file(ts_input=ts_row, lar_input=lar_frame, name="passes_10_rows.txt") #writes created file to disk
lar_frame

,record_id,lei,uli,app_date,loan_type,loan_purpose,preapproval,const_method,occ_type,loan_amount,...,aus_code_5,aus_result_1,aus_result_2,aus_result_3,aus_result_4,aus_result_5,aus_code_16,reverse_mortgage,open_end_credit,business_purpose
1,2,0PH53M7NA5F79002BWX8,0PH53M7NA5F79002BWX8TJ0E017DX6HFRJT4AZ9WM7D20,NA,3,32,2,2,1,12265,...,,17,,,,,,1,1,1
1,2,0PH53M7NA5F79002BWX8,0PH53M7NA5F79002BWX8T8VECSQYPUJ4VMPOSDWTVTB66,20180220,1,2,2,2,2,11266,...,L4G7Q4147AXG3YT2A9J8FSG2G3ZYJK5Z6H9S,2,10,12,8,,,1,2,1
1,2,0PH53M7NA5F79002BWX8,0PH53M7NA5F79002BWX8VPWODHPHWGO55LV0TUHCU6H76,NA,4,2,2,2,1,13736,...,,17,,,,,,2,1,1
1,2,0PH53M7NA5F79002BWX8,0PH53M7NA5F79002BWX8RCV5I1L19TOVIGHJW24XGDA79,20180407,4,4,2,2,1,19160,...,,9,,,1,,,2,1,2
1,2,0PH53M7NA5F79002BWX8,0PH53M7NA5F79002BWX8UXE817LF67RTWMF6Y9F9DVW79,NA,2,1,2,2,1,3407,...,,17,,,,,,2,2,1


In [12]:
file_maker = test_data(ts_schema_df, lar_schema_df)
ts_data, lar_data = utils.read_data_file(path="../edits_files/", data_file="passes_10_rows.txt")
file_maker.load_data_frames(ts_data, lar_data)
file_maker.s300_1_file()
file_maker.s300_2_file()

      data_type   dtypes_      field    valid_vals
0       numeric     [int]  record_id           [2]
1  alphanumeric  [string]        lei            []
2  alphanumeric  [string]        uli            []
3  alphanumeric  [string]   app_date          [NA]
4       numeric     [int]  loan_type  [1, 2, 3, 4]


In [16]:
validator.read_data_file(path="../edits_files/syntax/", data_file="s300_2.txt")
validator.reset_results()
#check data
for func in dir(validator):
    if func[:1] in ("s", "v") and func[1:4].isdigit()==True:
        #print("applying:", func)
        getattr(validator, func)()

#get validation results
results_df = pd.DataFrame(validator.results)
results_df[results_df.status=="failed"]

,edit_name,fail_count,fields,row_ids,row_type,status
1,s300_2,5.0,record_id,[0PH53M7NA5F79002BWX8TJ0E017DX6HFRJT4AZ9WM7D20...,LAR,failed
